In [ ]:
!pip install onnxruntime

In [ ]:
from transformers import AutoFeatureExtractor
import onnxruntime
import pickle
from scipy.special import softmax
from PIL import Image
from torchvision.transforms import ToTensor

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224",
                                                        do_rescale=False)

In [3]:
img_size = 224
img_path = """PATH TO IMAGE"""

In [4]:
with open('/path/to/classes.pkl', 'rb') as f:
    classes = pickle.load(f)

In [5]:
filepath = '/path/to/ViT-skin_demo.onnx'
ort_session = onnxruntime.InferenceSession(filepath)
input_name = ort_session.get_inputs()[0].name

In [21]:
def inference(img_path):

    img = Image.open(img_path)
    img_resized = img.resize((224, 224))
    img_resized = ToTensor()(img_resized)
    encoding = feature_extractor(images=img_resized, return_tensors='pt')

    ort_inputs = {input_name: encoding['pixel_values'].numpy()}
    ort_outs = ort_session.run(None, ort_inputs)

    result = softmax(ort_outs)
    result_dict = {}

    for i, value in enumerate(result[0][0].tolist()):
        result_dict[classes[str(i)]] = value
        result = sorted(result_dict.items(), key=lambda x:x[1], reverse=True)

    return result

In [22]:
import time
s = time.time()
result = inference(img_path)
e = time.time()

print(f'inference takes {e-s} sec')

inference takes 0.4013957977294922 sec


In [23]:
for i, (k, v) in enumerate(result[:3]):
    print(f"{k}일 확률 {int(v*100)}%")

무증상일 확률 50%
A2_비듬_각질_상피성잔고리일 확률 38%
A4_농포_여드름일 확률 10%
